In [2]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
# %load_ext line_profiler # Example: %lprun -T lprof0 -f celeri.celeri.get_okada_displacements u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1.values[segment_idx], segment.lat1[segment_idx], segment.lon2[segment_idx], segment.lat2[segment_idx], segment.locking_depth[segment_idx], segment.burial_depth[segment_idx], segment.dip[segment_idx], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)

In [3]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [4]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [11]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Build all linear operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Triangles here
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
# plt.figure()
# plt.imshow(np.log10(np.abs(operators.tri_station)), aspect="auto")
# plt.show()

# Get additiona matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

Calculating Okada partials for segments:   0%|          | 0/837 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Calculating cutde partials for triangles:   0%|          | 0/1841 [00:00<?, ?it/s]

/Users/meade/opt/anaconda3/envs/celeri/lib/python3.9/site-packages/cutde/coordinators.py:77: UserWarning: The obs_pts input array has Fortran ordering. Converting to C ordering. This may be expensive.
  warnings.warn(
/Users/meade/opt/anaconda3/envs/celeri/lib/python3.9/site-packages/cutde/coordinators.py:77: UserWarning: The obs_pts input array has Fortran ordering. Converting to C ordering. This may be expensive.
  warnings.warn(
/Users/meade/opt/anaconda3/envs/celeri/lib/python3.9/site-packages/cutde/coordinators.py:77: UserWarning: The obs_pts input array has Fortran ordering. Converting to C ordering. This may be expensive.
  warnings.warn(
/Users/meade/opt/anaconda3/envs/celeri/lib/python3.9/site-packages/cutde/coordinators.py:77: UserWarning: The obs_pts input array has Fortran ordering. Converting to C ordering. This may be expensive.
  warnings.warn(
/Users/meade/opt/anaconda3/envs/celeri/lib/python3.9/site-packages/cutde/coordinators.py:77: UserWarning: The obs_pts input arra

In [6]:
# Plot TDE displacements from a single element
tri_idx = 0
vel_east, vel_north, vel_up = celeri.get_tri_displacements(station.lon.to_numpy(), station.lat.to_numpy(), meshes, command.material_lambda, command.material_mu, tri_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0)
plt.figure()
plt.plot(meshes[0].lon1[tri_idx], meshes[0].lat1[tri_idx], ".r")
plt.plot(meshes[0].lon2[tri_idx], meshes[0].lat2[tri_idx], ".r")
plt.plot(meshes[0].lon3[tri_idx], meshes[0].lat3[tri_idx], ".r")
plt.quiver(station.lon, station.lat, vel_east, vel_north, scale=1e-3)
plt.show()

/Users/meade/opt/anaconda3/envs/celeri/lib/python3.9/site-packages/cutde/coordinators.py:77: UserWarning: The obs_pts input array has Fortran ordering. Converting to C ordering. This may be expensive.
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
def get_shared_sides(vertices):
    """
    Determine the indices of the triangular elements sharing 
    one side with a particular element.  
    Inputs:
    vertices: n x 3 array containing the 3 vertex indices of the n elements,
        assumes that values increase monotonically from 1:n

    Outputs:
    share: n x 3 array containing the indices of the m elements sharing a
        side with each of the n elements.  "-1" values in the array
        indicate elements with fewer than m neighbors (i.e., on
        the edge of the geometry).

    In general, elements will have 1 (mesh corners), 2 (mesh edges), or 3 
    (mesh interiors) neighbors, but in the case of branching faults that 
    have been adjusted with mergepatches, it's for edges and corners to 
    also up to 3 neighbors.
    """
    # Make side arrays containing vertex indices of sides
    side_1 = np.sort(np.vstack((vertices[:, 0], vertices[:, 1])).T, 1)
    side_2 = np.sort(np.vstack((vertices[:, 1], vertices[:, 2])).T, 1)
    side_3 = np.sort(np.vstack((vertices[:, 0], vertices[:, 2])).T, 1)
    sides_all = np.vstack((side_1, side_2, side_3))

    # Find the unique sides - each side can part of at most 2 elements
    _, first_occurence_idx = np.unique(sides_all, return_index=True, axis=0)
    _, last_occurence_idx = np.unique(np.flipud(sides_all), return_index=True, axis=0)
    last_occurence_idx = sides_all.shape[0] - last_occurence_idx - 1

    # Shared sides are those whose first and last indices are not equal
    shared = np.where((last_occurence_idx - first_occurence_idx) != 0)[0]

    # These are the indices of the shared sides
    sside1 = first_occurence_idx[shared] # What should I name these variables?
    sside2 = last_occurence_idx[shared]
                
    el1, sh1 = np.unravel_index(sside1, vertices.shape, order='F')  # "F" is for fortran ordering.  What should I call this variables?
    el2, sh2 = np.unravel_index(sside2, vertices.shape, order='F')
    share = -1 * np.ones((vertices.shape[0], 3))
    for i in range(el1.size):
        share[el1[i], sh1[i]] = el2[i]
        share[el2[i], sh2[i]] = el1[i]
    share = share.astype(int)
    return share

share = get_shared_sides(meshes[0].verts)

In [9]:
def get_tri_shared_sides_distances(share, x_centroid, y_centroid, z_centroid):
    """
    Calculates the distances between the centroids of adjacent triangular
    elements, for use in smoothing algorithms.

    Inputs:
    share: n x 3 array output from ShareSides, containing the indices
        of up to 3 elements that share a side with each of the n elements.
    x_centroid: x coordinates of element centroids
    y_centroid: y coordinates of element centroids
    z_centroid: z coordinates of element centroids

    Outputs:
    dists: n x 3 array containing distance between each of the n elements
        and its 3 or fewer neighbors.  A distance of 0 does not imply
        collocated elements, but rather implies that there are fewer 
        than 3 elements that share a side with the element in that row.
    """
    tri_shared_sides_distances = np.zeros(share.shape)
    for i in range(share.shape[0]):
        share[i, np.where(share[i, :] == -1)[0]] = i
        tri_shared_sides_distances[i, :] = np.sqrt((x_centroid[i] - x_centroid[share[i, :]])**2.0 + (y_centroid[i] - y_centroid[share[i, :]])**2.0 + (z_centroid[i] - z_centroid[share[i, :]])**2.0)
    return tri_shared_sides_distances

# TODO: Move centroid calculation to initial mesh read in???
meshes[0].x_centroid = (meshes[0].x1 + meshes[0].x2 + meshes[0].x3) / 3.0
meshes[0].y_centroid = (meshes[0].y1 + meshes[0].y2 + meshes[0].y3) / 3.0
meshes[0].z_centroid = (meshes[0].z1 + meshes[0].z2 + meshes[0].z3) / 3.0
tri_shared_sides_distances = get_tri_shared_sides_distances(share, meshes[0].x_centroid, meshes[0].x_centroid, meshes[0].x_centroid)

In [38]:
import copy
import scipy

# def get_tri_smoothing_matrix(share, tri_shared_sides_distances):
"""
Produces a smoothing matrix based on the scale-dependent
umbrella operator (e.g., Desbrun et al., 1999; Resor, 2004).

Inputs:
share: n x 3 array of indices of the up to 3 elements sharing a side 
    with each of the n elements
tri_shared_sides_distances: n x 3 array of distances between each of the
    n elements and its up to 3 neighbors

Outputs:
smoothing matrix: 3n x 3n smoothing matrix
"""

# Allocate sparse matrix for contructing smoothing matrix
n_shared_tris = share.shape[0]
smoothing_matrix = scipy.sparse.coo_matrix((3 * n_shared_tris, 3 * n_shared_tris))

# Create a design matrix for Laplacian construction
share_copy = copy.deepcopy(share)
share_copy[np.where(share != 0)[0]] = 1

# Sum the distances between each element and its neighbors
share_distances = np.sum(tri_shared_sides_distances, axis=1)
leading_coefficient = 2.0 / share_distances

# Replace zero distances with 1.  TODO: Do I need a copy.deepcopy here?
tri_shared_sides_distances[np.where(tri_shared_sides_distances == 0)] = 1

# Take the reciprocal of the distances.  TODO: Do I need a copy.deepcopy here?
inverse_tri_shared_sides_distances = 1.0 / tri_shared_sides_distances

# Diagonal terms # TODO: Defnitely not sure about his line!!!
# selfs = -lcoeff.*sum(idists.*s, 2);
diagonal_terms = -leading_coefficient * np.sum(inverse_tri_shared_sides_distances * np.vstack((share_distances, share_distances, share_distances)).T, 1)

# Off diagonal terms
off_diagonal_terms = repmat(lcoeff, 1, 3).*idists.*s;

# Place the weights into the smoothing operator
# for j = 1:3
# 	for i = 1:n
# 		w(3*i-(3-j), 3*i-(3-j))	= selfs(i)
# 		if share(i, j) ~= 0
# 			k = 3 * i - [2 1 0];
# 			m = 3 * share(i, j) - [2 1 0]
# 			p = sub2ind(size(w), k, m)
# 			w(p) = offdi(i, j)
# 		end
# 	end
# end	
# return w

ValueError: operands could not be broadcast together with shapes (1841,3) (1841,) 

In [49]:
a = np.vstack((share_distances, share_distances, share_distances)).T
inverse_tri_shared_sides_distances * a

array([[2.78012153e+00, 1.56188241e+00, 1.98298156e+04],
       [3.58695612e+00, 1.38664306e+00, 2.17538558e+04],
       [3.74323361e+00, 4.59773069e+00, 1.94041869e+00],
       ...,
       [1.28942925e+00, 2.51822701e+04, 4.45586413e+00],
       [1.58734633e+00, 1.84656091e+04, 2.70296860e+00],
       [2.88545937e+00, 4.55220378e+00, 2.30541752e+00]])

In [15]:
scipy.sparse.coo_matrix((4, 5))

<4x5 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in COOrdinate format>